# Running SignalP and annotating

In [1]:
import sys
import os
import glob
import re
import pandas as pd
import time

workDir = '/home/sam/FullCyc_metagenome/Other_studies_comp/redo_analysis/signalp_output'

dbCAN_db = '/home/sam/new_databases/dbCAN/HMM/dbCAN-fam-HMMs.txt'
MEROPS_db = '/home/sam/new_databases/MEROPS/peptidase3/MEROPS_protease.dmnd'
ESTHER_db = '/home/sam/new_databases/ESTHER/HMMs/hydrolaseDB'

nproc = 50

## Running SignalP

In [2]:
def run_signalp(study, genome):
    os.system(' '.join(['signalp', 
                        '-fasta', os.path.join('/home/sam/FullCyc_metagenome/Other_studies_comp/redo_analysis/prokka_output', study, genome, genome + '.prokka.faa'),
                        '-format short',
                        '-batch 1000',
                        '-org gram+',
                        '-prefix', os.path.join(workDir, study, 'signalp_out', genome + '_signalp_gram_pos')]))
    
    os.system(' '.join(['signalp', 
                        '-fasta', os.path.join('/home/sam/FullCyc_metagenome/Other_studies_comp/redo_analysis/prokka_output', study, genome, genome + '.prokka.faa'),
                        '-format short',
                        '-batch 1000',
                        '-org gram-',
                        '-prefix', os.path.join(workDir, study, 'signalp_out', genome + '_signalp_gram_neg')]))

### RefSoil

In [ ]:
study = 'RefSoil'
genome_list = [fna for fna in os.listdir('/home/sam/new_databases/RefSoil/RefSoil_genomes/genome_fasta') if fna.endswith('.fna')]
genome_list

for genome in genome_list:
    run_signalp(study, genome)



In [ ]:
gram_pos_merge = pd.DataFrame()

gram_pos_list = [f for f in os.listdir(os.path.join(workDir, study, 'signalp_out')) if re.search('gram_pos', f)]


for SP_file in gram_pos_list:
    output_df = pd.read_csv(os.path.join(workDir, study, 'signalp_out', SP_file), sep='\t', skiprows=[0], header=None)
    new_header = output_df.iloc[0] 
    output_df = output_df[1:]
    output_df.columns = new_header
    output_df['genome'] = re.sub('_signalp_gram_pos_summary.signalp5', '', SP_file)
    gram_pos_merge = gram_pos_merge.append(output_df)
    output_df = None
    
print(len(gram_pos_merge['genome']))

gram_pos_merge.to_csv(os.path.join(workDir, study, 'signalp_gram_pos_summary.signalp5'), 
                      index=False, sep='\t')

gram_pos_merge = None


In [ ]:
gram_neg_merge = pd.DataFrame()

gram_neg_list = [f for f in os.listdir(os.path.join(workDir, study, 'signalp_out')) if re.search('gram_neg', f)]


for SP_file in gram_neg_list:
    output_df = pd.read_csv(os.path.join(workDir, study, 'signalp_out', SP_file), sep='\t', skiprows=[0], header=None)
    new_header = output_df.iloc[0] 
    output_df = output_df[1:]
    output_df.columns = new_header
    output_df['genome'] = re.sub('_signalp_gram_neg_summary.signalp5', '', SP_file)
    gram_neg_merge = gram_neg_merge.append(output_df)
    output_df = None
    
print(len(gram_neg_merge['genome']))

gram_neg_merge.to_csv(os.path.join(workDir, study, 'signalp_gram_neg_summary.signalp5'), 
                      index=False, sep='\t')

gram_neg_merge = None
